```{r}
#| eval: false
#| echo: false

library(knitr)

# useful function for options
`%||%` <- function(l, r) {
  if (is.null(l)) r else l
}

# super-customised table printing ----
knit_print.data.frame <- function (x, options, ...) {
  # get options
  digits <- options$digits %||% getOption("digits")
  rownames <- options$rownames %||% FALSE
  pageLength <- options$pageLength %||% 10 
  escape <- options$escape %||% TRUE
  caption <- options$table.cap 
  
  # use DT for longer tables in html
  if (nrow(x) > pageLength & knitr::is_html_output()) {
    numeric_cols <- sapply(x, is.numeric) |> which() |> names()
    dt <- DT::datatable(x, 
                        rownames = rownames,
                        caption = caption,
                        escape = escape,
                        width = "100%",
                        height = "auto",
                        options = list(pageLength = pageLength),
                        selection = "none")
    if (length(numeric_cols) > 0) {
      dt <- DT::formatRound(dt, 
                            columns = numeric_cols,
                            digits = digits)
    }
    knitr::knit_print(dt, options)
  } else {
    # use kableExtra::kable for PDFs or shorter tables
    k <- kableExtra::kable(x, 
                      digits = digits, 
                      row.names = rownames, 
                      caption = caption,
                      escape = escape) |>
      kableExtra::kable_styling(
        full_width = options$full_width,
        bootstrap_options = c("striped", "hover")
      )
    
    if (knitr::is_html_output()) {
        k <- c("<div class=\"kable-table\">", k, "</div>") |>
          paste(collapse = "\n")
    }
    
    knitr::asis_output(k)
  }
}
registerS3method("knit_print", "data.frame", knit_print.data.frame)
```



## Using Inkscape Extensions


```{r}
#| echo: false
#| eval: false
inx_extensions_list_win <- function(){
  path = system('inkscape --system-data-directory', intern = TRUE)
  inkscape_extensions_path = paste(path, "\\extensions", sep = "")
  inkscape_python_home  = paste(gsub("\\share\\inkscape", "", path, fixed = T), "\\bin", sep = "")
  inkscape_extension_names = list.files(path = inkscape_extensions_path, pattern = ".py")
  df <- data.frame(Extension = inkscape_extension_names, Description = NA)
  for (i in c(1:nrow(df))) {
    inkscape_extension_name = df$Extension[i]
    print(inkscape_extension_name)
    output = tempfile("inx_", fileext = ".txt")
    con  = tempfile(pattern = "inx_", fileext = ".bat")
    fmt = '@ECHO OFF
cd %s
python.exe "%s" --help  > %s'
    text = sprintf(fmt,
                   inkscape_python_home,
                   paste(inkscape_extensions_path, inkscape_extension_name, sep = "\\"),
                   output)
    writeLines(text, con)
    system(con)
    output
    inkscape_extension_description <- readLines(output)
    df$Description[i] <- paste(inkscape_extension_description, collapse = "\n")
  }

  df
}

inx_extensions_list_win() %>% readr::write_csv("./data/extensions.csv")
```

```{r}
#| echo: false
#| warning: false
#| message: false
"./data/extensions.csv" |>
  readr::read_csv() 
```


### Function `inx_extension()`

![](https://upload.wikimedia.org/wikipedia/commons/3/30/Den_Haag_wapen.svg)


In [ ]:
using Printf, Pipe, ArchGDAL, DataFrames, GeoDataFrames, Plots

function is_url(x)
  re = r"^https?://"
  occursin(re, x)
end

function inx_actions(input, actions, ext)
  input_file_path = tempname() * ".svg"
  if is_url(input)
    download(input, input_file_path)
  else
    cp(input, input_file_path)
  end
  inkscape = "inkscape"
  output = tempname() * ext
  command = "--shell" # or --batch-process
  actions = @sprintf "%s export-filename:%s;export-do" actions output
  actions = "--actions=" * actions
  read(`$inkscape $command $actions $input_file_path`, String)
  output
end

function inx_extension(input, inkscape_extension_name, ext)
  input_file_path = tempname() * ".svg"
  if is_url(input)
    download(input, input_file_path)
  else
    cp(input, input_file_path)
  end
  inkscape = "inkscape"
  command = "--system-data-directory"
  path = read(`$inkscape $command`, String)
  path = replace(path, "\n"=>"")
  inkscape_python_home  = replace(path, "\\share\\inkscape"=>"") * "\\bin"
  
  inkscape_extension_path = path * "\\extensions\\" * inkscape_extension_name 
  output = tempname() * ext
  bat = tempname() * ".bat"
  text = @sprintf "@ECHO OFF \n cd \"%s\" \n python.exe \"%s\" --output=\"%s\" \"%s\"\n" inkscape_python_home inkscape_extension_path output input_file_path
  open(bat, "w") do file
    write(bat, text)
  end
  run(`$bat`)
  output
  #replace(output, "\\"=>"//")
end

In [ ]:
df = @pipe "https://upload.wikimedia.org/wikipedia/commons/3/30/Den_Haag_wapen.svg" |>
  svg2dxf(_) |> 
  ArchGDAL.read(_) |>
  ArchGDAL.getlayer(_, 0) |>
  DataFrame(_) |> 
  _.""  |>
  DataFrame(geometry=_)
df.geometry |> plot

#  |> 
#  _.""  |>
#  DataFrame(geometry=_)

## Using Inkscape Actions


```{r}
#| echo: false
#| eval: false
inx_actions_list_win <- function(){
  output = tempfile("inx_", fileext = ".txt")
  con  = tempfile(pattern = "inx_", fileext = ".bat")
  fmt = '@ECHO OFF
inkscape --action-list > %s'
  text = sprintf(fmt, output)
  writeLines(text, con)
  system(con)
  text <- readLines(output)
  text <- gsub("<b0>", "\u00B0", x = text, useBytes = F, fixed = T)
  #dd[agrep("object-rotate-90-ccw", dd, useBytes = F, fixed = T)]
  writeLines(text, output)
  actions <- readr::read_delim(output, delim = ":", col_names = FALSE)
  names(actions) = c("Action", "Description")
  actions
}
inx_actions_list_win() %>% readr::write_csv("./data/actions.csv")
```

```{r}
#| echo: false
#| warning: false
#| message: false
"./data/actions.csv" |> 
  readr::read_csv()
```


## Inkscape.jl


In [ ]:
df = @pipe "https://upload.wikimedia.org/wikipedia/commons/1/1f/Julia_Programming_Language_Logo.svg" |>
  inx_actions(_, "file-open;select-all;transform-scale:10;page-fit-to-selection;", ".svg") |>
  inx_extension(_, "dxf12_outlines.py", ".dxf") |> 
  ArchGDAL.read(_) |>
  ArchGDAL.getlayer(_, 0) |>
  DataFrame(_) |> 
  _.""  |>
  DataFrame(geometry=_)

function make_poly(x)
  res =  ArchGDAL.createlinestring()
  for i in 1:nrow(x)
    res = ArchGDAL.union(res, x[i, "geometry"])
  end
  res = ArchGDAL.polygonize(res)
  vec = [ArchGDAL.createpolygon()] # to have proper type
  for i in 0:(ArchGDAL.ngeom(res) - 1)
    push!(vec, ArchGDAL.getgeom(res, i)) 
  end
  DataFrame(geometry=vec[2:length(vec)])
end 

pdf = @pipe df |> make_poly(_)
@pipe pdf |> GeoDataFrames.write("./sig/sig.shp", _)

#import Pkg; Pkg.add(["ArchGDAL", "DataFrames", "Pipe", "GeoDataFrames", "Luxor"])

# raw plot
df.geometry |> plot

# polygon plot
pdf.geometry |> plot

In [ ]:
using Shapefile, Luxor
include(joinpath(dirname(pathof(Luxor)), "readshapefiles.jl"))

hue = ["#000000", "#4d64ae", "#000000", "#000000", "#ca3c32", "#9259a3", "#399746", "#000000", "#000000", "#000000", "#FFFFFF"]

function draw_sig(x)
  sig = open(x) do f
    read(f, Shapefile.Handle)
  end
  Drawing(4000, 2000, "./sig/sig.png")
  origin(Point(500, 1900))
  scale(7, 7)
  gsave()
  background("#FFFFFF")
  i = 1
  for shape in sig.shapes
      pgons, bbox = convert(Array{Luxor.Point, 1}, shape)
      for pgon in pgons
          sethue(hue[i])
          poly(pgon, :fill)
          i = i + 1
      end
  end
  grestore()
#  sethue("black")
  finish()
#  preview()
end

draw_sig("./sig/sig.shp")

![](./sig/sig.png)
